# Traveling Salesman Problem (TSP)

**The Traveling Salesman Problem (TSP)** is a famous optimization problem in computer science and operations research. It involves finding the shortest possible route for a salesman to visit a set of cities exactly once and return to the starting point. The problem is **NP-hard**, meaning that there is no known efficient algorithm to solve it optimally for large instances. This has led to the development of approximation algorithms that provide near-optimal solutions in a reasonable amount of time.

Note the difference between **Hamiltonian Cycle and TSP**. The Hamiltonian cycle problem is to find if there exists a tour that visits every city exactly once. Here we know that a Hamiltonian tour exists (because the graph is complete), and in fact, many such tours exist; **the problem is to find a minimum weight Hamiltonian cycle.**

For this project, I focus on the **Christofides Algorithm**, which was independently developed by **Nikolai Serdyukov** in 1978. This approximation algorithm is specifically designed for the **metric TSP**, where the triangle inequality holds (i.e., going directly from \(A\) to \(C\) is never more expensive than going via \(B\)). The assumption of metric properties is not just a theoretical convenience but a reflection of many real-world problems, such as road networks, where direct routes are typically shorter or equal in length compared to detours. Metric TSP simplifies the problem structure and allows approximation algorithms like Christofides-Serdyukov to achieve guaranteed performance within **1.5 times the optimal solution**, making it both efficient and reliable for practical applications in logistics, delivery systems, and network design.

In [ ]:
#importing necessary libraries
import numpy as np
import networkx as nx
from collections import defaultdict

**Class for generating matrices**

In [781]:
class DistanceMatrixGenerator:
    @staticmethod
    def generate_metric_matrix(size, max_coordinate=100, min_distance=1):
        """
        Generate a symmetric distance matrix that satisfies the triangle inequality (metric TSP).
        This function will repeatedly generate matrices until one satisfies the conditions.
        :param size: Number of cities (vertices).
        :param max_coordinate: Maximum coordinate value for the cities.
        :param min_distance: Minimum distance between cities.
        :return: Symmetric distance matrix with integer weights.
        """
        while True:  # Loop until a valid matrix is generated
            coordinates = np.random.randint(0, max_coordinate, size=(size, 2))
            matrix = np.zeros((size, size), dtype=int)

            # Calculate distances and populate the matrix
            for i in range(size):
                for j in range(size):
                    if i != j:
                        distance = np.linalg.norm(coordinates[i] - coordinates[j])
                        matrix[i][j] = max(int(round(distance)), min_distance)

            if DistanceMatrixGenerator.validate_metric_properties(matrix):
                return matrix  # Return the valid matrix

    @staticmethod
    def validate_metric_properties(matrix):
        """
        Validate that the matrix satisfies the triangle inequality.
        :param matrix: Distance matrix.
        :return: True if metric properties hold, False otherwise.
        """
        size = len(matrix)
        for i in range(size):
            for j in range(size):
                for k in range(size):
                    if i != j and j != k and i != k:
                        if matrix[i][k] > matrix[i][j] + matrix[j][k]:
                            return False
        return True

**Class for handling Multigraphs**

In [774]:
class MultiGraph:
    def __init__(self, num_vertices):
        self.graph = defaultdict(lambda: defaultdict(int))
        self.num_vertices = num_vertices

    def add_edge(self, u, v, weight):
        """
        Add an edge to the multigraph.
        """
        self.graph[u][v] += 1
        self.graph[v][u] += 1

    def get_degree(self, vertex):
        """
        Get the degree of a vertex, accounting for duplicate edges.
        """
        return sum(self.graph[vertex].values())

    def to_adjacency_matrix(self):
        """
        Convert the multigraph to a standard adjacency matrix.
        """
        matrix = np.zeros((self.num_vertices, self.num_vertices), dtype=int)
        for u in self.graph:
            for v in self.graph[u]:
                matrix[u][v] = self.graph[u][v]
        return matrix

**Class for finding eulerian tour - modified**

In [787]:
class GraphDFS:
    def __init__(self, adjacency_matrix):
        """
        Initialize the graph with an adjacency matrix.
        :param adjacency_matrix: 2D list or numpy array representing the adjacency matrix.
        """
        self.V = len(adjacency_matrix)
        self.graph = {i: {} for i in range(self.V)}
        self.convert_matrix_to_list(adjacency_matrix)

    def convert_matrix_to_list(self, adjacency_matrix):
        """
        Convert the adjacency matrix into an adjacency list.
        """
        for i in range(self.V):
            for j in range(self.V):
                if adjacency_matrix[i][j] > 0:
                    self.graph[i][j] = adjacency_matrix[i][j]

    def is_connected(self):
        """
        Check if the graph is connected using DFS.
        """
        visited = [False] * self.V
        start = next((i for i in range(self.V) if self.graph[i]), -1)
        if start == -1:
            return True
        self.dfs(start, visited)
        return all(visited[i] or not self.graph[i] for i in range(self.V))

    def dfs(self, v, visited):
        """
        Perform DFS to mark visited vertices.
        """
        visited[v] = True
        for neighbor in self.graph[v]:
            if not visited[neighbor]:
                self.dfs(neighbor, visited)

    def has_all_even_degree(self):
        """
        Check if all vertices have even degrees.
        """
        return all(sum(self.graph[v].values()) % 2 == 0 for v in self.graph)

    def dfs_eulerian(self, v, path):
        """
        Perform DFS to construct the Eulerian cycle in a multigraph.
        """
        while self.graph[v]:
            neighbor = next(iter(self.graph[v]))
            self.graph[v][neighbor] -= 1
            self.graph[neighbor][v] -= 1
            if self.graph[v][neighbor] == 0:
                del self.graph[v][neighbor]
            if self.graph[neighbor][v] == 0:
                del self.graph[neighbor][v]
            self.dfs_eulerian(neighbor, path)
        path.append(v)

    def euler_tour(self):
        """
        Construct the Eulerian cycle starting from the first vertex.
        """
        if not self.is_connected() or not self.has_all_even_degree():
            raise ValueError("The graph does not meet the conditions for an Eulerian cycle.")

        path = []
        for i in range(self.V):
            if self.graph[i]:  # Start from a vertex with edges
                self.dfs_eulerian(i, path)
                break

        path.reverse()
        return [(path[i], path[i + 1]) for i in range(len(path) - 1)]

**Class with Christofides algorithm**

In [792]:
class Christofides:
    def __init__(self, adjacency_matrix):
        """
        Initialize the Christofides algorithm with the adjacency matrix.
        :param adjacency_matrix: 2D list or numpy array representing the distance matrix.
        """
        self.adjacency_matrix = np.array(adjacency_matrix, dtype=int)
        self.num_vertices = len(adjacency_matrix)
        self.mst = MultiGraph(self.num_vertices)
        self.eulerian_graph = None
        self.final_tour = []

    def find_mst(self):
        """
        Find the Minimum Spanning Tree (MST) of the graph.
        """
        G = nx.Graph()
        for i in range(self.num_vertices):
            for j in range(i + 1, self.num_vertices):
                if self.adjacency_matrix[i][j] > 0:
                    G.add_edge(i, j, weight=self.adjacency_matrix[i][j])
        mst_edges = list(nx.minimum_spanning_tree(G).edges(data=True))
        for u, v, data in mst_edges:
            self.mst.add_edge(u, v, data["weight"])

    def find_odd_degree_vertices(self):
        """
        Find vertices with odd degrees in the MST.
        """
        return [i for i in range(self.num_vertices) if self.mst.get_degree(i) % 2 != 0]

    def find_min_weight_matching(self, odd_vertices):
        """
        Find a minimum weight perfect matching among odd-degree vertices and add it to the MST.
        """
        G = nx.Graph()
        for i in odd_vertices:
            for j in odd_vertices:
                if i != j:
                    G.add_edge(i, j, weight=self.adjacency_matrix[i][j])
        matching = nx.algorithms.matching.min_weight_matching(G)
        for u, v in matching:
            self.mst.add_edge(u, v, self.adjacency_matrix[u][v])

    def validate_eulerian_graph(self):
        """
        Validate that the graph has all even degrees after matching.
        """
        degrees = [self.mst.get_degree(i) for i in range(self.num_vertices)]
        if not all(degree % 2 == 0 for degree in degrees):
            raise ValueError("The graph does not have all even degrees after matching.")

    def find_eulerian_cycle(self):
        """
        Construct an Eulerian cycle from the graph.
        """
        self.validate_eulerian_graph()
        adj_matrix = self.mst.to_adjacency_matrix()
        graph_dfs = GraphDFS(adj_matrix)
        eulerian_cycle = graph_dfs.euler_tour()
        if not eulerian_cycle:
            raise ValueError("Eulerian cycle construction failed.")
        return eulerian_cycle

    def shortcut_tour(self, eulerian_cycle):
        """
        Construct a Hamiltonian cycle from the Eulerian cycle by visiting each vertex exactly once.
        """
        visited = set()
        tour = []
        for u, v in eulerian_cycle:
            if u not in visited:
                tour.append(u)
                visited.add(u)
        if tour[0] != tour[-1]:
            tour.append(tour[0])
        self.final_tour = tour

    def solve(self):
        """
        Solve the TSP using Christofides' algorithm.
        :return: Hamiltonian cycle (final tour).
        """
        self.find_mst()
        odd_vertices = self.find_odd_degree_vertices()
        self.find_min_weight_matching(odd_vertices)
        eulerian_cycle = self.find_eulerian_cycle()
        self.shortcut_tour(eulerian_cycle)
        return self.final_tour

    def calculate_tour_cost(self):
        """
        Calculate the total cost of the TSP tour.
        :return: Total cost of the Hamiltonian cycle.
        """
        if not self.final_tour:
            raise ValueError("No Hamiltonian cycle found.")
        return sum(
            self.adjacency_matrix[self.final_tour[i]][self.final_tour[i + 1]]
            for i in range(len(self.final_tour) - 1)
        )

**Example usage**

In [799]:
matrix3 = [[ 0, 10,  7,  9, 11],
       [10,  0,  5,  2,  2],
       [ 7,  5,  0,  5,  5],
       [ 9,  2,  5,  0,  4],
       [11,  2,  5,  4,  0]]

christofides = Christofides(matrix3)
try:
       solution = christofides.solve()
       cost = christofides.calculate_tour_cost()
       print("Final Tour:", solution)
       print("Total Cost:", cost)
except Exception as e:
       print(f"Error: {e}")

Final Tour: [0, 2, 1, 4, 3, 0]
Total Cost: 27


**Example usage 2**

In [800]:
sample_matrix = DistanceMatrixGenerator.generate_metric_matrix(size=15, max_coordinate=10)
sample_matrix

array([[ 0,  4,  5,  7,  7,  9,  2,  9,  1,  8,  9,  5,  1,  8,  4],
       [ 4,  0,  2,  6,  4,  5,  2,  6,  3,  6,  7,  4,  4,  5,  1],
       [ 5,  2,  0,  4,  2,  4,  3,  5,  5,  4,  5,  6,  5,  3,  2],
       [ 7,  6,  4,  0,  3,  6,  5,  8,  7,  1,  2,  9,  7,  4,  6],
       [ 7,  4,  2,  3,  0,  3,  5,  5,  7,  3,  3,  8,  7,  1,  4],
       [ 9,  5,  4,  6,  3,  0,  7,  2,  8,  6,  6,  7,  9,  3,  5],
       [ 2,  2,  3,  5,  5,  7,  0,  8,  2,  6,  7,  5,  2,  6,  2],
       [ 9,  6,  5,  8,  5,  2,  8,  0,  9,  8,  8,  7,  9,  5,  6],
       [ 1,  3,  5,  7,  7,  8,  2,  9,  0,  8,  9,  4,  1,  8,  3],
       [ 8,  6,  4,  1,  3,  6,  6,  8,  8,  0,  1, 10,  8,  3,  6],
       [ 9,  7,  5,  2,  3,  6,  7,  8,  9,  1,  0, 11,  9,  3,  7],
       [ 5,  4,  6,  9,  8,  7,  5,  7,  4, 10, 11,  0,  5,  9,  4],
       [ 1,  4,  5,  7,  7,  9,  2,  9,  1,  8,  9,  5,  0,  8,  4],
       [ 8,  5,  3,  4,  1,  3,  6,  5,  8,  3,  3,  9,  8,  0,  5],
       [ 4,  1,  2,  6,  4,  5,  2

In [801]:
christofides = Christofides(sample_matrix)
try:
    solution = christofides.solve()
    cost = christofides.calculate_tour_cost()
    print("Final Tour:", solution)
    print("Total Cost:", cost)
except Exception as e:
    print(f"Error: {e}")

Final Tour: [0, 6, 1, 2, 4, 3, 9, 10, 13, 5, 7, 11, 14, 8, 12, 0]
Total Cost: 37
